In [19]:
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
import re

In [13]:
proj_path = "/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand"
class_path = os.path.join(proj_path,"Thailand_classification")
ts_path = os.path.join(proj_path,"Thailand_download_timeseries")
# pd.read_csv("
os.listdir(class_path)

loc_id = 0

s2_csv_name = f"pt_ts_loc{loc_id}_s2.csv"
s2_csv_name



'pt_ts_loc0_s2.csv'

In [23]:
os.listdir(class_path)

['countif.xlsx', 'location_classification.csv']

In [112]:
class s2Dataset(Dataset):
    """Sentinel 2 dataset"""
    
    def __init__(self, proj_path, class_colname):
        """
        Args:
            proj_path (string): path to manclassify project
        """
        self.proj_path = proj_path
        proj_normpath = os.path.normpath(proj_path)
        proj_dirname = proj_normpath.split(os.sep)[-1]
        self.proj_name = re.sub("_classification$","",proj_dirname)
        self.class_path = os.path.join(proj_path, self.proj_name + "_classification")
        self.ts_path = os.path.join(proj_path, self.proj_name + "_download_timeseries")
        self.pt_classes = pd.read_csv(os.path.join(self.class_path,"location_classification.csv"))
        self.pt_classes = classes[['loc_id', class_colname]].dropna()
        self.classes = pd.unique(self.pt_classes[class_colname])
    
    def __getitem__(self, idx):
        return 10
        
    def __len__(self):
        return self.pt_classes.shape[0]

In [113]:
s2_dataset = s2Dataset(proj_path = proj_path, class_colname = 'Subclass2019')
classes = pd.read_csv(os.path.join(s2_dataset.class_path,"location_classification.csv"))
# c2 = classes[['loc_id', 'Subclass2019']]#[!classes['Subclass2019'].isnan]
s2_dataset.__len__()

501

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [29]:
proj_path

'/Users/gopal/Google Drive/_Research/Research projects/ML/manclassify/app_data/Thailand'